# CMRM Assignment No. 2

In [ ]:
import os
import librosa
import scipy
import sklearn # pip install sklearn
import numpy as np
import pandas as pd # pip install pandas
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm # pip install tqdm
from sklearn.linear_model import LogisticRegression
from mutagen.easyid3 import EasyID3 # pip install mutagen

## Question 1

In [ ]:
train_path = 'sonics/train'
test_path = 'sonics/test'

# -------------------------------------------------- #
# Train set
fake_path = train_path + '/fake'
real_path = train_path + '/real'

fake_train = [i for i in os.listdir(fake_path) if i[0] != '.']
fake_train.sort()

# Print the number of files
print("Number of fake tracks in the train set:", len(fake_train))


real_train = [i for i in os.listdir(real_path) if i[0] != '.'] #read the real songs contained in the folder
real_train.sort() #sort the names
#print(real_train)

# Print the number of files
print("Number of real tracks in the train set:", len(real_train))



# -------------------------------------------------- #
# Test set
test_set = [i for i in os.listdir(test_path) if i[0] != '.'] #read the files in path
test_set.sort()
# for filename in tqdm(test_set):
#     path = os.path.join(test_path, filename)
#     print(path)

# Print the number of files
print("Number of tracks in the test set:", len(test_set))

In [ ]:
# Load signal fake_train[0] - Time domain
Fs = 22050
# Fs_fake = 22050 / 2
# Fs_real = 44100
def print_plot_play(x, Fs, text=''):
    print('%s Fs = %d, x.shape = %s, x.dtype = %s' % (text, Fs, x.shape, x.dtype))
    plt.figure(figsize=(10, 2))
    plt.plot(x, color='gray')
    plt.xlim([0, x.shape[0]])
    plt.xlabel('Time (samples)')
    plt.ylabel('Amplitude')
    plt.tight_layout()
    plt.show()
    ipd.display(ipd.Audio(data=x, rate=Fs))
    


#Fake track 1
fake_mp3_1_path = os.path.join(fake_path, fake_train[0]) #get full path
fake_mp3_1, _ = librosa.load(fake_mp3_1_path, sr=Fs) #load using librosa
print_plot_play(x=fake_mp3_1, Fs=Fs, text='First Fake MP3: ') #plot the first fake song

#Real track 1
real_mp3_1_path = os.path.join(real_path, real_train[0])
print(real_train[0])
real_mp3_1, _ = librosa.load(real_mp3_1_path, sr=Fs)

print_plot_play(x=real_mp3_1, Fs=Fs, text='First Real MP3: ')

In [ ]:
# FFT
#use np.fft
#Fake song 1
X = np.abs(np.fft.fft(fake_mp3_1))
N = fake_mp3_1.shape[0] #number of samples
freq = np.fft.fftfreq(N, d=1/Fs)
X = X[:N//2]
freq = freq[:N//2]

print("Fake song 1 Frequency Spectrum")
plt.figure(figsize=(15, 2))
plt.plot(freq, X, c='k')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()

#Real song 1
X = np.abs(np.fft.fft(real_mp3_1)) #take absolute value of fourier transform to get positive amplitude values of frequencies
N = real_mp3_1.shape[0] #number of samples
freq = np.fft.fftfreq(N, d=1/Fs) #get respective frequency axis, fftfreq returns the Discrete Fourier Transform sample frequencies to use as the x-axis
X = X[:N//2] #remove redundant mirror duplicate produced by FFT
freq = freq[:N//2]

print("Real song 1 Frequency Spectrum")
plt.figure(figsize=(15, 2))
plt.plot(freq, X, c='k')
#plt.xlim([20, 10000])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()

# STFT
N = 4096
H = 1024

#just for reference but unnecessary because of librosa.stft
# def stft_basic(x, w, H=8):
#     """Compute a basic version of the discrete short-time Fourier transform (STFT)
#
#     Args:
#         x: Signal to be transformed
#         w: Window function
#         H: Hopsize
#
#     Returns:
#         X: The discrete short-time Fourier transform
#     """
#     N = len(w)
#     L = len(x)
#     M = np.floor((L - N) / H).astype(int)
#     X = np.zeros((N, M + 1), dtype='complex')
#     for m in range(M + 1):
#         x_win = x[m * H:m * H + N] * w
#         X_win = np.fft.fft(x_win)
#         X[:, m] = X_win
#     K = (N + 1) // 2
#     X = X[:K, :]
#     return X

# w = np.hanning(N)
# X_fake = stft_basic(fake_mp3_1, w, H)
# X_real = stft_basic(real_mp3_1, w, H)

#Compute STFT of the 1st real and fake song examples by using librosa stft function
X_fake = librosa.stft(fake_mp3_1, n_fft=N, hop_length=H, win_length=N, window='hann')
X_real = librosa.stft(real_mp3_1, n_fft=N, hop_length=H, win_length=N, window='hann')

#helper function to plot spectrogram output of stft function
def plot_power_spectrogram(x, X, N, Fs):
    Y = np.abs(X) ** 2
    eps = np.finfo(float).eps

    Y_db = 10 * np.log10(Y + eps)
    real_t = np.arange(len(x)) / Fs

    plt.figure(figsize=(12, 4))

    plt.plot(real_t, x)
    plt.xlim([min(real_t), max(real_t)])

    time_axis = np.arange(X.shape[1]) * H / Fs
    frequency_axis = np.arange(X.shape[0]) * Fs / N

    left = min(time_axis)
    right = max(time_axis) + N / Fs
    lower = min(frequency_axis)
    upper = max(frequency_axis)
    plt.figure(figsize=(12, 4))
    im = plt.imshow(Y_db, origin='lower', aspect='auto', cmap='viridis', extent=[left, right, lower, upper], vmin=-30,
                    vmax=20)
    plt.colorbar(im)
    plt.ylim([0, 5000])
    plt.xlabel('Time (seconds)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plt.show()


#Plot power "Spectrogram"...not frequency spectrum graph
print("Fake song 1 STFT Power Spectrogram")
plot_power_spectrogram(fake_mp3_1, X_fake, N, Fs)

#Plot real
print("Real song 1 STFT Power Spectrogram")
plot_power_spectrogram(real_mp3_1, X_real, N, Fs)


## Question 2

In [ ]:
from utils import lower_envelope, max_normalise, curve_profile

#Testing functions:

#fake_mp3_1_norm = max_normalise(fake_mp3_1, 0.5)
# real_mp3_1_norm = max_normalise(real_mp3_1, 0.5)
# #
# # fake_mp3_1_lower_env, hull = lower_envelope(fake_mp3_1)
# #
# print_plot_play(x=real_mp3_1_norm, Fs=Fs, text='First Real MP3 Clipped: ')
# # print_plot_play(x=fake_mp3_1_lower_env, Fs=Fs, text='First Fake MP3 low env: ')
#
#
# print(abs(X_fake)**2)
# X_fake_abs = np.abs(X_fake)
# spectral_curve_fake = np.mean(X_fake_abs, axis=1)
# print("Shape of the new spectral curve:", spectral_curve_fake.shape)
# print(f"dimensions: {spectral_curve_fake.ndim}")
#
# # Plot the average spectrum for the fake track
# xfreqs_fake1 = librosa.fft_frequencies(sr=Fs, n_fft=N)
# plt.figure(figsize=(12, 4))
# plt.plot(xfreqs_fake1, spectral_curve_fake)
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("Average Magnitude")
# plt.title("Average Spectral Curve (Fake Track)")
# plt.show()
#
# spectral_curve_fake_db = librosa.amplitude_to_db(spectral_curve_fake)
# print(f"dimensions: {spectral_curve_fake_db.ndim}")
# xdim, fake_x_curve = curve_profile(xfreqs_fake1, spectral_curve_fake_db)
# plt.figure(figsize=(12, 4))
# plt.plot(xdim, max_normalise(fake_x_curve))
# plt.xlabel("Frequency (Hz)")
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("Average Magnitude")
# plt.title("Average Spectral Curve (Fake Track)")
# plt.show()
#
# spectral_curve_real = np.mean(np.abs(X_real), axis=1)
# spectral_curve_real_db = librosa.amplitude_to_db(spectral_curve_real, ref=np.max)
# freqs = librosa.fft_frequencies(sr=Fs, n_fft=N)
#
# x_profile_real, y_profile_real = curve_profile(x=freqs, c=spectral_curve_real_db)
#
# plt.figure(figsize=(12, 4))
# plt.plot(x_profile_real, max_normalise(y_profile_real))
# plt.title("Spectral Profile (Real Track)")
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("Magnitude Above Envelope (dB)")
# plt.grid(True)
# plt.show()

def fakeprint(stft, f_range = [0, 16000], fs = 22050):
    """
    Compute the fakeprint feature from an STFT representation.

        Args:
            stft: 2D STFT spectrogram.
            f_range: Frequency range [min_freq, max_freq] over which the
                     curve profile is computed.
            SR: Sampling rate of the original audio signal.

        Returns:
            fp_curve: Normalized fakeprint feature curve extracted from the
                      averaged spectral profile.
    """

    N = 4096

    stft_abs = np.abs(stft) #need absolute value of stft output from librosa (same as fft)
    spectral_curve = np.mean(stft_abs, axis=1) #compute the average of the stft across the time axis to get a 1D curve
    xfreqs = librosa.fft_frequencies(sr=fs, n_fft=N) #x axis
    # print(len(xfreqs))
    spectral_curve_db = librosa.amplitude_to_db(spectral_curve) #convert to decibels
    x_freqs, x_curve = curve_profile(x=xfreqs, c=spectral_curve_db, f_range=f_range)
    # print(len(x_curve))
    # print(len(x_freqs))
    fp_curve = max_normalise(x_curve) #normalize to ensure quiet and louder songs are represented equally

    return fp_curve

In [ ]:
# Test fakeprint
# print(Fs)
# print(N)

f_range = [0, 16000]
curve_freqs = librosa.fft_frequencies(sr=Fs, n_fft=N)

#Slice frequency axis to match output length of fakeprint curve
cutoff_idx = np.where( (f_range[0] < curve_freqs) & (curve_freqs < f_range[1] ))
x_ = curve_freqs[cutoff_idx]


fake_fp_curve = fakeprint(X_fake, fs=Fs)
print(len(fake_fp_curve))
plt.figure(figsize=(12, 4))
plt.plot(x_, fake_fp_curve)
plt.xlabel("Frequency (Hz)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("Fake Track Curve Profile")
plt.show()

real_fp_curve = fakeprint(X_real)
plt.figure(figsize=(12, 4))
plt.plot(x_, real_fp_curve)
plt.title("Real Track Curve Profile")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()


## Question 3

In [ ]:
def compute_feature_vector(filename, Fs, N=4096, H=1024):
    """Compute the feature vector 

    Args:
        filename: Input filename
        Fs: Sampling rate
        N: Window length
        H: Hop size

    Returns:
        f_vector: Feature vector
    """
    #assuming filename is the full path to the file
    x, _ = librosa.load(filename, sr=Fs)
    X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, window='hann')
    f_vector = fakeprint(X)

    
    return f_vector

In [ ]:
#Function test
f_vector_fake1 = compute_feature_vector(fake_mp3_1_path, Fs=Fs)
f_range = [0, 16000]
curve_freqs = librosa.fft_frequencies(sr=Fs, n_fft=N)

#Slice frequency axis to match output length of fakeprint curve
cutoff_idx = np.where( (f_range[0] < curve_freqs) & (curve_freqs < f_range[1] ))
x_ = curve_freqs[cutoff_idx]
#check function output
plt.plot(x_, f_vector_fake1)
plt.xlabel("Frequency (Hz)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Average Magnitude")
plt.title("Compute Feature Vector Check: Average Spectral Curve (Fake Track)")
plt.show()

# Define labels_dict
labels_dict = {0: 'fake', 1: 'real'}

# Compute feature vector - train
real_train_f_vectors = []
train_vectors_list = []

#labels list
labels_list = []

#compute feature vectors of real songs from the training set and append 1 to labels list
for filename in tqdm(real_train):
    path = os.path.join(real_path, filename)
    f_vector = compute_feature_vector(path, Fs=Fs)
    real_train_f_vectors.append(f_vector)
    train_vectors_list.append(f_vector)
    labels_list.append(1)

#compute feature vectors of fake songs from the training set and append 0 to labels list
fake_train_f_vectors = []
for filename in tqdm(fake_train):
    path = os.path.join(fake_path, filename)
    f_vector = compute_feature_vector(path, Fs=Fs)
    fake_train_f_vectors.append(f_vector)
    train_vectors_list.append(f_vector)
    labels_list.append(0)

#convert to numpy array
train_vectors = np.array(train_vectors_list)
labels = np.array(labels_list)

print(f"shape of train set: {train_vectors.shape}")

# plt.plot(x_, fake_train_f_vectors[0])
# plt.xlabel("Frequency (Hz)")
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("Average Magnitude")
# plt.title("Average Spectral Curve (Fake Track)")
# plt.show()

# Compute feature vector - test
test_vectors_list = []
for filename in tqdm(test_set):
    path = os.path.join(test_path, filename)
    f_vector = compute_feature_vector(path, Fs=Fs)
    test_vectors_list.append(f_vector)

test_vectors = np.array(test_vectors_list)
print(f"shape of test set: {test_vectors.shape}")



In [ ]:
# Train regressor
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(train_vectors, labels)
logreg.fit(train_vectors, labels)
predictions = logreg.predict(train_vectors)

#Changing threshold for determining real song (use for End of Question 3)
threshold = 0.7
probs = logreg.predict_proba(train_vectors) #get the "confidence" number between 0-1 first that a song is fake then if it's real, this number must be above the threshold set above to be classified as "fake" or "real" respectively (default is 5).
probability_real_song = probs[:, 1] #get only the confidence numbers if a song is "real"
#print(probs[:,1])

custom_predictions = (probability_real_song > threshold).astype(int) #swap for predictions variable below to implement the custom threshold

In [ ]:
# Print the accuracy for the train set
accuracy = logreg.score(train_vectors, labels)
print(f"Training Accuracy: {accuracy}")

# Plot the confusion matrix
conf_matrix = confusion_matrix(labels, predictions)
matrix_plot = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['fake', 'real'])

matrix_plot.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()


## Question 4

In [ ]:
# Compute prediction for the test set
test_set_predictions = logreg.predict(test_vectors)
# Print tracks classified as 'real'
real_tracks = []
print('The real tracks are: \n')
for filename,pred in zip(test_set, test_set_predictions):
    if pred == 1:
        real_tracks.append(filename)
        print(filename)


In [ ]:
# Find password
psw = 0

for filename in real_tracks:
    path = os.path.join(test_path, filename)
    print(path)
    track_metadata = EasyID3(path) #extracts track metadata given file path as input
    try:
        print(track_metadata['date'])
    except KeyError:
        print(f"No date metadata found for current file: {filename}")
        continue
    psw += int(track_metadata['date'][0])

print(f'The password is: {psw}')

In [ ]:
# Verify if your items are real
test_ground_truth = pd.read_csv('sonics/ground_truth.csv')
print(f"No. of columns: {len(test_ground_truth.columns)}")
print(f"No. of rows: {len(test_ground_truth.index)}")

print("The column names (head) are: ")
print(*test_ground_truth.columns, sep=', ')

#print(test_ground_truth.loc[test_ground_truth['id'] == 51890, 'label'].values[0])

#print(test_ground_truth.loc[test_ground_truth['id'] == '51890', 'label'].values[0])
print("\n")
print("Reading the track label of the determined real tracks: ")
for filename in real_tracks:
    id = filename.split('.')[0]
    #print(id)
    #print(f"Filename: {filename} is actually {test_ground_truth.loc[test_ground_truth['id'] == int(id), 'label'].values[0]}")
    print(f"Filename {filename} has label: {test_ground_truth.loc[test_ground_truth['id'] == int(id), 'label'].values[0]}")



In [ ]:
# Print the accuracy for the test set
test_set_labels_list = []
for filename in test_set:
    id = filename.split('.')[0]
    if test_ground_truth.loc[test_ground_truth['id'] == int(id), 'label'].values[0] == 'real':
        print(id)
        test_set_labels_list.append(1)
    else:
        test_set_labels_list.append(0)

test_set_labels = np.array(test_set_labels_list)
print(f"No. of labels: {len(test_set_labels)}")
test_accuracy = logreg.score(test_vectors, test_set_labels)
print(f"Test Accuracy: {test_accuracy}")



# Plot the confusion matrix
test_confusion_matrix = confusion_matrix(test_set_labels, test_set_predictions)
test_matrix_plot = ConfusionMatrixDisplay(confusion_matrix=test_confusion_matrix, display_labels=['fake', 'real'])

test_matrix_plot.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

#compute weighted coefficients of how much each frequency of the fakeprint curve affects the decision
#print(logreg.coef_)
weight_coeffs = logreg.coef_[0]
#print(weight_coeffs)

frequency_axis = x_ #get relevant frequency fft axis computed earlier from np.fftfreq

#plot logreg coefficient weights vs frequency
plt.figure(figsize=(12, 4))
plt.plot(frequency_axis, weight_coeffs)
plt.title("LogReg Coefficients")
plt.xlabel("Frequency")
plt.ylabel("Coefficient Weight")
plt.show()

## Question 5

In [ ]:
# Find the key sentence
key_sentence = ''
for filename in real_tracks:
    id = filename.split('.')[0]
    #print(id)
    key_sentence+= test_ground_truth.loc[test_ground_truth['id'] == int(id), 'lyrics_feature'].values[0]
    path = os.path.join(test_path, filename)
    ipd.display(ipd.Audio(path, rate=Fs))

print(key_sentence)